In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
import matplotlib.animation as animation
from matplotlib import rc
from sklearn.datasets import make_blobs
from random import sample
from random import random
import random

### Función para graficar a partir de una gráfica y la lista de thetas para la animación

In [ ]:
def animacion(fig, thetas):
    x = np.linspace(ax.get_xlim()[0], ax.get_xlim()[1], 200)
    def anima(i):            
        y = thetas[i][0] +thetas[i][1] * x
        line.set_data(x, y)
        return line,
    return animation.FuncAnimation(fig, anima, frames=len(thetas), interval=100, blit=True)


# Perceptrón
Dado un conjunto linealmente separable el algoritmo del perceptrón encuentra
las thetas de la recta que permiten clasificar las observaciones a partir de un
conjunto de entrenamiento.
$\\ $
$X \in M_{MxN}$ donde $M$ es el número de observaciones y $N$ el número de características.$\\ $
$y \in R^M $ donde $y_i = 1$ si la observación $i$ pertenece a la clase o $y_i = -1$ si no.$\\ $
$\tau$ es el número de repeticiones para que el algoritmo converja



In [ ]:
def perceptron(X, y, tao = 100):
    M, N =  X.shape # M - Filas, N - Columnas
    theta = np.zeros(N)
    theta0 = np.zeros(1)        

    thetas = []
    for i in range(tao):        
        for j in range(M):
            # Si es menor a 0 significa que está mal clasificado
            if np.dot(y[j], np.dot(theta, X[j][:]) + theta0) <= 0:
                theta = theta + y[j]*X[j][:]
                theta0 = theta0 + y[j]                
                thetas.append([-theta0/theta[1],-theta[0]/theta[1]])                
        
    return (theta, theta0, thetas)

In [ ]:
def test(X, y, opcion, perc = 0.8, lr = 0.01, lamb = 0.1):
    n = int(len(y)*perc)    
    M, N = X.shape

    # Índices aleatorios para el conjunto de entrenamiento
    index = sample([i for i in range(len(y))], n) 
    
    #Filtramos el conjunto de entrenamiento
    mask = np.zeros(len(y), dtype=bool)
    mask[index] = True
    trainingSetX = X[mask,]
    trainingSetY = y[mask]        

    #Filtramos el resto de los índices para el conjunto de evaluación  
    mask = np.ones(len(y), dtype = bool)
    mask[index] = False
    testSetX = X[mask,]
    testSetY = y[mask]
    
    # Dependiendo de la opción usa el algoritmo
    if opcion == "Perceptron" : 
        theta, theta0, thetas = perceptron(trainingSetX, trainingSetY)   
    elif opcion == "SVM":
        theta, theta0, thetas = gradientDescentSVM(trainingSetX, trainingSetY, np.array([np.random.rand() for i in range (N)]), np.random.rand(), lr, lamb = lamb)
    else:
        thetaEst, thetasEst = gradientDescentEst(X, Y, learning_rate = lr, lamb = lamb)
    
    #Evaluamos el modelo con las thetas que nos regresó el modelo
    correct = [0 if testSetY[i]*(np.dot(theta, testSetX[i,:])+theta0) <= 0 else 1 for i in range(len(testSetY))]
    print("Accuracy of " + str(sum(correct)/len(testSetY)*100) + "%")
    return (theta, theta0, thetas)


# Prueba de perceptrón

In [ ]:
iris = load_iris()
# Cargamos datos
X = iris.data
Y = iris.target
# Filtramos entre setosa y versicolor
X = X[Y<2, 0:2]
Y = Y[Y<2]
#Lo modificamos para que donde sea setosa Y = 1
# y -1 en c. o. c. para que funcione el perceptrón
Y = np.where(Y == 0, 1, -1)

In [ ]:
theta, theta0, thetas = perceptron(X,Y, tao = 200)

# Animacióm del perceptrón

In [ ]:
fig, ax = plt.subplots()

line, = ax.plot([], [], 'k-')

ax.scatter(X[Y==1, 0], X[Y==1, 1], c='b', label=iris.target_names[0])
ax.scatter(X[Y==-1, 0], X[Y==-1, 1], c='r', label="Not setosa")
ax.legend(loc='lower right')

ax.set_xlabel(iris.feature_names[0])
ax.set_ylabel(iris.feature_names[1])

x = np.linspace(ax.get_xlim()[0], ax.get_xlim()[1], 200)
ax.scatter(x, -theta0/theta[1] -theta[0]/theta[1]*x)

ax.grid(True)

In [ ]:
fig, ax = plt.subplots()

line, = ax.plot([], [], 'k-')

ax.scatter(X[Y==1, 0], X[Y==1, 1], c='b', label=iris.target_names[0])
ax.scatter(X[Y==-1, 0], X[Y==-1, 1], c='r', label="Not setosa")
ax.legend(loc='lower right')

ax.set_xlabel(iris.feature_names[0])
ax.set_ylabel(iris.feature_names[1])


ax.grid(True)
animado = animacion(fig, thetas)

rc('animation', html='jshtml')
animado

# Descenso por gradiente del clasificador lineal de máximo margen 

## Función de pérdida
$ L_h(\nu) = \begin{cases} 
    \text{1-v}& \text{si }\nu < 1 \\
    \text{0}&  c. o. c.
\end{cases}$

In [ ]:
def hingeLoss(v):    
    return(np.where(v<1, 1-v, 0))

## Derivada de la función de pérdida
$ L_h(\nu) = \begin{cases} 
    \text{-1}& \text{si }\nu < 1 \\
    \text{0}&  c. o. c.
\end{cases}$

In [ ]:
def dHingeLoss(v):
    return(np.where(v<1, -1, 0))

## Función de costo
$J(\theta, \theta_0) = \frac{1}{n} \displaystyle\sum_{i = 1}^{n} Loss_h(y^{(i)}(\theta\cdot x^{(i)} + \theta_0))$$+ \frac{\lambda}{2}\lVert \theta \rVert^2$

In [ ]:
def J(theta, theta0, X, y, lamb = 0.001):
    n = len(y)
    suma = 1/n*np.sum(hingeLoss(y*(np.dot(X, theta.transpose())+theta0))) + lamb/2*np.dot(theta, theta)
    return suma


In [ ]:
J(theta, theta0, X, Y)

## Derivada de la función de costo
$\nabla_\theta J= \frac{1}{n} \displaystyle\sum_{i = 1}^{n} Loss_h^\prime(y^{(i)}(\theta\cdot x^{(i)} + \theta_0))y^{(i)}x^{(i)}$$+ \lambda \theta$

In [ ]:
def dJ(theta, theta0, X, y, lamb = 0.001):
    n = len(y)
    suma =  1/n*np.sum(np.dot(dHingeLoss(y*(np.dot(X, theta.transpose())+theta0)), y)*X, axis = 0) + lamb*theta
    return suma

In [ ]:
dJ(theta, theta0, X, Y)

## Derivada de la función de costo con respecto a $\theta_0$
$\frac{\partial J}{\partial \theta_0}= \frac{1}{n} \displaystyle\sum_{i = 1}^{n} Loss_h^\prime(y^{(i)}(\theta\cdot x^{(i)} + \theta_0))y^{(i)}$

In [ ]:
def dJ_0 (theta, theta0, X, y):
    n = len(y)
    suma =  1/n*np.sum(np.dot(dHingeLoss(y*(np.dot(X, theta.transpose())+theta0)), y))
    return suma

In [ ]:
dJ_0(theta, theta0, X, Y )

In [ ]:
def gradientDescentSVM(X, y, theta_, theta0_, learningRate, iter = 500, eps = 1e-6, lamb = 0.01):
    theta = theta_
    theta0 = theta0_
    n = len(y)
    t = 0   
    thetas = []
    # Mientras no converge o no alcanza el número máximo de iteraciones se mantiene ejecutándose
    while True and t < iter:
        t = t+1
        thetaT = theta - learningRate*dJ(theta, theta0, X, y)
        theta0T = theta0 - learningRate*dJ_0(theta, theta0, X, y)
        if abs(J(theta, theta0, X, y)- J(thetaT, theta0T, X, y)) <  eps:
            break
        theta = thetaT
        theta0 =  theta0T
        #y = -theta0/theta[1] + (-theta[0]/theta[1])*x
        thetas.append([-theta0/theta[1],-theta[0]/theta[1]])
    return (thetaT, theta0T, thetas)


In [ ]:
thetaSVM, theta0SVM, thetasSVM = gradientDescentSVM(X, Y, theta , theta0 , 0.01)

In [ ]:
animado = animacion(fig, thetasSVM)

rc('animation', html='jshtml')
animado

# Descenso por gradiente estocástico

In [ ]:
def gradientDescentEst(X, y, learning_rate, T = 1000, lamb = 0.01):
    M,N = X.shape # M - Filas, N - Columnas
    random.seed(1)
    theta = np.array([np.random.rand() for i in range (N)])  
    thetas = []
    for t in range (T):
        index = random.randint(0, M-1)             
        theta = theta - learning_rate*((np.dot(theta.transpose(), X[index,:])-y[index:index+1])*X[index,:]+lamb/M*theta) 
        thetas.append([0, -theta[0]/theta[1]])            
    return (theta, thetas)


In [ ]:
thetaEst, thetasEst = gradientDescentEst(X,Y, 0.01)

In [ ]:
animado = animacion(fig, thetasEst[250:750])

rc('animation', html='jshtml')
animado

# Cross validation

In [ ]:
def crossValidation(X, y, k, lamb):
    M, N =  X.shape # M - Filas, N - Columnas
    size = int(M/k )
    error = 0
    theta = np.array([np.random.rand() for i in range (N)])
    theta0 = np.random.rand()            
    
    for i in range(k):
        leftX = X[0:size*i, :]
        rightX = X[size*(i+1): M , :]     
        leftY = y[0:size*i]
        rightY = y[size*(i+1): M ]   
        # gradientDescentSVM(X, y, theta_, theta0_, learningRate, iter = 200, eps = 1e-6, lamb = 0.01)
        theta,theta0,t_ = gradientDescentSVM(np.concatenate((leftX, rightX)), np.concatenate((leftY, rightY)), theta, theta0, 0.01,           lamb = lamb)         
        testingSetX = X[size*i:size*(i+1), :]
        testingSetY = y[size*i:size*(i+1)]        
        error += sum([1 if np.dot(y[j], np.dot(theta, X[j][:]) + theta0) <= 0 else 0 for j in range(size) ])    
    return 1/k*error
    

In [ ]:
error = crossValidation(X, Y, 5, 0.001)
error


# Prueba de los modelos y selección de lambda

In [ ]:
# Creamos un conjunt del 80% par entrenar y 20% para probar
n = int(len(Y)*0.8)    
index = sample([i for i in range(len(Y))], n)     
    
mask = np.zeros(len(Y), dtype=bool)
mask[index] = True
trainingSetX = X[mask,]
trainingSetY = Y[mask]        

mask = np.ones(len(Y), dtype = bool)
mask[index] = False
testSetX = X[mask,]
testSetY = Y[mask] 


In [ ]:
# PELIGRO: NO CORRER ESTA CELDA, tarda bastante tiempo
errores = []
lambdas = []

for i in range (1, 10000):
    lambdas.append (1/i)
    #errores.append (crossValidation(trainingSetX, trainingSetY, 5, 1/i))
    errores.append (crossValidation(X, Y, 5, 1/i))    

In [ ]:
plt.scatter(errores, lambdas) 

plt.xlabel("Errores")
plt.ylabel("Lambdas")

# Análisis 
Al observar la gráfica nos dimos cuenta que una lambda muy chica hacía que los errores fueran mayores que con una más grande.
Nosotros pensamos que con una lambda muy pequeña hay overfitting al conjunto de entrenamiento y por eso hay un error más grande con el conjunto de prueba.
En ese sentido, decidimos seleccionar una lambda pequeña $\lambda = 0.1$, pero no tanto para evitar este fenomeno.

In [ ]:
# Tomamos lambda = 0.1
thetaSVM, theta0SVM, thetasSVM = test(X, Y, "SVM", lr = 0.001, lamb = 0.1)

In [ ]:
theta, theta0, thetas = test(X,Y, "Perceptron")

# Conclusión
Ambos modelos tienen en general una buena precisión, pero a veces es baja. Sin embargo, después de las diferentes pruebas, el método más estable es el del descenso por gradiente(SVM), pues al observar la animación nos damos cuenta que el Perceptrón varía mucho a veces cuando se corrigen las theta's.$\\ $
Asimismo, el método que se nos hizo más interesante fue el descenso por gradiente estocástico. Lo anterior debido a que, a pesar de que en cada iteración solo corrige el modelo con una observación, al final converge a resultados considerablemente buenos.

